In [ ]:
import aisuite as ai
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Create an instance of the AISuite client
client = ai.Client()

prompt = "What is the weather in New York?"
messages = [
    {
        "role": "user",
        "content": prompt,
    }
]
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=messages,


)

# See the LLM response
print(response.choices[0].message.content)

In [ ]:
import requests
import json
from geopy.geocoders import Nominatim


def current_weather(place_name:str):

    geolocator = Nominatim(user_agent="my_unique_app_name_1234")
    location = geolocator.geocode(place_name)
    if location:

        url = "https://api.open-meteo.com/v1/forecast"

        # Set parameters: ask for current weather
        params = {
            "latitude": location.latitude, 
            "longitude": location.longitude,
            "current_weather": "true",     # this gives real-time data
            # optionally, you can ask for other data (hourly, daily), e.g.:
            # "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m"
        }

        # Make the request
        response = requests.get(url, params=params)
        data = response.json()
        return data
    

data = (current_weather("New York"))
print(json.dumps(data, indent=4))


{
    "latitude": 40.710335,
    "longitude": -73.99309,
    "generationtime_ms": 0.1131296157836914,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 32.0,
    "current_weather_units": {
        "time": "iso8601",
        "interval": "seconds",
        "temperature": "\u00b0C",
        "windspeed": "km/h",
        "winddirection": "\u00b0",
        "is_day": "",
        "weathercode": "wmo code"
    },
    "current_weather": {
        "time": "2025-11-16T17:00",
        "interval": 900,
        "temperature": 11.6,
        "windspeed": 39.0,
        "winddirection": 291,
        "is_day": 1,
        "weathercode": 1
    }
}


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "current_weather",
            "description": "temperature at a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "place_name": {
                        "type": "string",
                        "description": "The place to look for weather, e.g. Heraklion",
                    },
#                     "longitude": {
#                         "type": "string",
#                         "description": "The longtitude of the location, e.g. 74.0060",
#                     },
                },
                "required": ["place_name"],
            },
        },
    }
]

In [ ]:
user_prompt = "What is the temperature in New York"

import json

completion = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    tools=tools,
    tool_choice="auto",  # specify the function call
)

print(completion.choices[0].message)
# print(completion.choices[0].message.tool_calls[0].function)
params =completion.choices[0].message.tool_calls[0].function.arguments
params=json.loads(params)
print((params))

In [ ]:
tool_call = completion.choices[0].message.tool_calls[0]
func_name = tool_call.function.name
func_args = json.loads(tool_call.function.arguments)
func_args,func_name

In [ ]:
chosen_function =  eval(func_name)
content = chosen_function(**params)
content['current_weather']

In [ ]:
# user_prompt = "What is the temperature in New York"
user_prompt = "Can i go with a t shirt in NewYork? is it windy or cold?"

second_completion = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[
        {"role": "user", "content": user_prompt},
        completion.choices[0].message,
        {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content":json.dumps(content['current_weather'])
        },
    ],
    tools=tools
)

print(second_completion.choices[0].message.content)